In [ ]:
from google.colab import files
uploaded = files.upload()

Saving DadosBerlim.xlsx to DadosBerlim.xlsx


In [ ]:
#IMPORTA AS BIBLIOTECAS
import random #Importa biblioteca randômica
import matplotlib.pyplot as plt #Importa biblioteca gráfica
import pandas as pd #Importa biblioteca para leitura do banco de dados
import io #Importa biblioteca para leitura do banco de dados
import math #Importa biblioteca matemática
from itertools import product #Importa biblioteca para criar iteradores
from scipy.spatial import distance #Importa biblioteca para caucular distância euclidiana
import numpy as np


#DEFINE AS VARIÁVEIS
alfa = 1 #fator atrelado ao feromônio
beta = 4 #fator atrelado à heurística
rho = 0.2 #Taxa de evaporação do feromônio
num_formigas = 52 #Quantidade de formigas (mesma quantidade de cidades)
Q = 100 #Quantidade de feromônio depositado
b = 10 #Elitismo
tau = [10**-6] * (((num_formigas)**2) - num_formigas)  #Vetor com taxa inicial de feromônio


#LEITURA E ARMAZENAMENTO DOS DADOS
df = pd.read_excel(io.BytesIO(uploaded["DadosBerlim.xlsx"])) #Leitura do arquivo excel

x = df["X"].values #Armazena a posição x das cidades
y = df["Y"].values #Armazena a posição y das cidades

pontos = [] #Inicializa o vetor para armazenar os pontos x e y de cada cidade

for i in range(num_formigas): #Itera todos as cidades
  if not (math.isnan(x[i]) or math.isnan(y[i])): #Verifica de o ponto x e y são números. Se forem,
    pontos.append((x[i], y[i])) #Cria uma tupla contendo um par de coordenadas correspondendo as cidades


#DEFINE A POSIÇÃO INICIAL DAS FORMIGAS
posicao_formigas = random.sample(pontos, num_formigas) #Seleciona a posição das formigas de maneira aleatória com base na localização (pontos) das cidades


#COMBINAÇÕES
combinacoes = list(product(pontos, repeat = 2)) #Armazena todas as combinações de 2 pontos (2 cidades) em um vetor
combinacoes_validas = [c for c in combinacoes if c[0] != c[1]] #Armazena somente as combinações onde as duas cidades existentes são diferentes uma da outra (distancia != 0)


#CALCULA A DISTÂNCIA E VISIBILIDADE ENTRE DOIS PONTOS
distancias = [] #Inicializa o vetor que armazena todas as distâncias entre 2 cidades
visibilidade =  [] #Inicializa o vetor que armazena o valor das visibilidades de cada caminho

for i in range(len(combinacoes_validas)): #Itera todas as combinações geradas
  dist = distance.euclidean(combinacoes_validas[i][0], combinacoes_validas[i][1]) #Calcula a distância entre os dois pontos
  distancias.append(dist) #Armazena as distâncias em um vetor

  visib = 1 / distancias[i] #Calcula a visibilidade como o inverso da distância (minimização)
  visibilidade.append(visib) #Armazena as visibilidades em um vetor

In [ ]:
#SELEÇÃO DOS CAMINHOS
melhores_distancias = [] #Inicializa o vetor de melhores distâncias (menores)
melhor_distancia = float('inf') #Inicializa a variável de melhor distância com um valor infinito, para que este seja atualizado na primeira iteração
melhor_rota = None #Inicializa a melhor rota
probabilidades = [0] * len(combinacoes_validas) #Inicializa o vetor que armazenará as probabilidades com zeros

num_iteracoes = 50
for geracao in range(num_iteracoes): #Itera o algoritmo

  for i in range(len(combinacoes_validas)): #Itera todas as combinações geradas
    probabilidades[i] = (tau[i] ** alfa) * (visibilidade[i] ** beta) #Calcula a probabilidade de se visitar cada cidade

  for i in range(len(probabilidades)): #Itera todas as probabilidades calculadas
    probabilidades[i] /= sum(probabilidades) #Divide-as pelo somatório, normalizando-as

  caminhos_selecionados = [] #Inicializa o vetor de caminhos selecionados por todas as formigas

  for i in range(num_formigas): #Itera todas as formigas
    caminhos_formiga = [] #Inicializa o vetor de caminho de uma formiga

    posicao_atual = posicao_formigas[i] #Inicializa a posição atual da formiga
    caminhos_visitados = set() #Cria um conjunto vazio para armazenar os caminhos já percorridos

    for j in range(len(pontos)): #Itera todas as cidades
      cidades_disponiveis = [(a, b) for (a, b) in combinacoes_validas if a == posicao_atual and b not in caminhos_visitados] #Percorre as combinações válidas, verifica as que se relacionam a posição atual e conferem se as cidades a serem visitadas ainda estão disponíveis
      probabilidades_formiga = [probabilidades[i] for i, comb in enumerate(combinacoes_validas) if comb in cidades_disponiveis] #Armazenas as probabilidades relacionadas ao vetor acima

      if not cidades_disponiveis: #Se não houverem cidades disponíveis
        break #O loop é interrompido

      caminho_selecionado = random.choices(cidades_disponiveis, weights = probabilidades_formiga, k = 1)[0] #Seleciona uma cidade disponível com base na probabilidade dela
      caminhos_formiga.append(caminho_selecionado) #Adiciona o caminho selecionado no vetor
      caminhos_visitados.add(caminho_selecionado[1]) #Adiciona as cidades já visitadas no vetor de controle (armazena o segundo elemento da tupla, onde a formiga parou)
      posicao_atual = caminho_selecionado[1] #Atualiza a posição atual da formiga (segundo elemento da tupla, onde a formiga parou)

    caminhos_selecionados.append(caminhos_formiga) #Adiciona o trajeto da formiga (individual) no vetor de todas as formigas


    #DISTÂNCIA FORMIGAS
    distancias_totais = [] #Inicializa o vetor de distância

    for caminhos_formiga in caminhos_selecionados: #Itera o trajeto de todas as formigas
      dist_tot = 0.0 #Inicializa a variável que armazenará a soma das distâncias

      for caminho in caminhos_formiga: #Itera ponto a ponto do trajeto de uma formiga
        dist_tot += distance.euclidean (caminho[0], caminho[1]) #Calcula a distância de cada ponto, somando-as
      distancias_totais.append(dist_tot) #Armazena o valor no vetor


    #FEROMÔNIO
    for k, caminhos_formiga in enumerate(caminhos_selecionados): #Para cada formiga, itera todos os itens de caminhos_selecionados
      for caminho in caminhos_selecionados[k]: #Para cada caminho
        aresta = caminho #Atribui o valor do caminho à variável aresta
        L = distancias_totais[k] #Atribui o valor da distância total do caminho à variável L
        delta_tau = Q / L #Calcula delta tau

        if distancias_totais[k] < melhor_distancia: #Se a distância total da formiga k for menor do que a melhor distância
          melhor_distancia = distancias_totais[k] #A variável de melhor_distância é atualizada
          melhor_rota = caminhos_selecionados[k] #A variável de melhor_rota é atualizada

        index_aresta = combinacoes_validas.index(aresta) #Atribui o indice da primeira aresta

        if caminho in caminhos_formiga: #Se caminho estiver presente em caminhos_formiga, ou seja, já tiver sido percorrido,
          tau[index_aresta] += b * delta_tau #delta tau é atualizado utilizando como reforço a variável b
        else: #Senão
          tau[index_aresta] += delta_tau #Apenas adiciona delta tau

    for l in range(len(tau)): #Para cada elemento dentro de tau
      tau[l] *= (1 - rho) #Realiza-se o processo de evaporação do feromônio

    menor_caminho = min(distancias_totais) #Encontra o menor caminho dentro do vetor de distancias_totais
  melhores_distancias.append(menor_caminho) #Armazena esse menor caminho no vetor de melhores distâncias


In [ ]:
menor_distancia_final = min(melhores_distancias) #Acha a distância mínima

#GRÁFICO DE MELHORES DISTÂNCIAS
plt.plot(melhores_distancias, label='alfa: {}\nbeta: {}\nrho: {}\nNumero de formigas: {}\nQ: {}\nb: {}\nQuantidade de Iterações: {}\nMenor Distância: {:.2f}'.format(alfa, beta, rho, num_formigas, Q, b, num_iteracoes, menor_distancia_final))
plt.xlabel('Iteração')
plt.ylabel('Menor distância')
plt.title('Menores distâncias encontradas ao longo das iterações')
plt.legend()
plt.show()


#GERA A LISTA COM AS COORDENADAS RELACIONADAS AO MELHOR CAMINHO
melhor_rota_coords = [caminho[0] for caminho in melhor_rota] + [melhor_rota[-1][1]]


#DEFINE AS COORDENADAS DAS CIDADES
x_cidades = [ponto[0] for ponto in pontos]
y_cidades = [ponto[1] for ponto in pontos]


#GRAFO DO MELHOR CAMINHO
plt.figure(figsize=(8, 6))
plt.scatter(x_cidades, y_cidades, color='blue', label='Cidades')
plt.plot([coord[0] for coord in melhor_rota_coords], [coord[1] for coord in melhor_rota_coords], color='red', linewidth=2, label='Menor caminho')
plt.xlabel('Coordenada x')
plt.ylabel('Coordenada y')
plt.title('Melhor Rota')
plt.legend()
plt.show()